In [1]:
%matplotlib inline
%time from hikyuu.interactive import *

2025-08-09 00:47:52,759 [INFO] runing in interactive session [<module>] (D:\workspace\hikyuu\hikyuu\__init__.py:109) [hikyuu::hku_info]
2025-08-09 00:47:52,760 [INFO] running in jupyter [<module>] (D:\workspace\hikyuu\hikyuu\__init__.py:116) [hikyuu::hku_info]


current plugin path: D:\workspace\hikyuu\hikyuu\plugin
2025-08-09 00:47:52.850 [HKU-I] - Using MYSQL BaseInfoDriver (BaseInfoDriver.cpp:57)
2025-08-09 00:47:52.878 [HKU-I] - 加载市场信息…… (StockManager.cpp:576)
2025-08-09 00:47:52.890 [HKU-I] - 加载证券类型信息…… (StockManager.cpp:594)
2025-08-09 00:47:52.896 [HKU-I] - 加载证券信息…… (StockManager.cpp:480)
2025-08-09 00:47:53.021 [HKU-I] - 加载权息数据…… (StockManager.cpp:612)
2025-08-09 00:47:54.019 [HKU-I] - 加载板块信息…… (StockManager.cpp:185)
2025-08-09 00:47:55.150 [HKU-I] - 加载K线数据…… (StockManager.cpp:189)
2025-08-09 00:47:55.150 [HKU-I] - 预加载 day K线数据至缓存 (最大数量: 100000)! (StockManager.cpp:233)
2025-08-09 00:47:55.151 [HKU-I] - 2.29 秒数据加载完毕. (StockManager.cpp:195)
2025-08-09 00:47:55.152 [HKU-I] - 启动行情接收. (SpotAgent.cpp:38)
CPU times: total: 2.02 s
Wall time: 4.74 s


# 1 利用 TM 实现简单的记账本

TradeManager对象可以理解为一个模拟的交易账户，负责交易的买/卖操作、记录交易记录以及持仓情况，也可以通过修改其买/卖操作的接口实现实盘接入。创建一个模拟交易账户，通常使用快捷创建函数 crtTM。TM对象的基本操作：

- buy  买入
- sell 卖出
- checkin 存入现金
- checkout 取出现金

可以利用 TM 实现简单的记账本，手工记录自己的操作情况，例如：

In [2]:
#创建一个初始资金10万元，起始日期2017年1月1日的模拟账户
my_tm = crtTM(init_cash=100000, date=Datetime(201701010000))

#2017年1月3日以9.11的价格买入100股
td = my_tm.buy(Datetime(201701030000), sm['sz000001'], 9.11, 100)

#查看当前资金及持仓情况
print(my_tm)

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-01-03 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current total funds: 100005.00,
  current cash: 99089.00,
  current market_value: 916.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
    SZ000001 平安银行 2017-01-03 00:00:00 2089 100.00 911.00 1240.00 329.00 36.11% 0.33%
  Short Position: 
  Borrow Stock: 
}


In [3]:
#转化为pandas的DataFrame显示当前持仓情况 
position = my_tm.get_position_list()
position.to_df()

,证券代码,证券名称,买入时间,持仓天数,持仓数量,投入金额,当前市值,盈亏金额,盈亏百分比,止损价,目标价,清仓时间,累计持有数量,累计成本,累计风险,累计买入金额,累计卖出金额
0,SZ000001,平安银行,2017-01-03,3140,100.0,911.0,1240.0,329.0,36.11,0.0,0.0,NaT,100.0,0.0,911.0,911.0,0.0


In [10]:
my_tm.get_trade_list().to_df()

,证券代码,证券名称,交易时间,业务名称,计划交易价格,实际成交价格,目标价格,交易数量,止损价,现金余额,交易总成本,佣金,印花税,过户费,其它成本,信号来源,备注
0,,,2017-01-01,INIT,300000.00,300000.00,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0,--,
1,SZ000001,平安银行,2024-12-31,BUY,11.93,11.93,NaN,1000.0,0.0,288070.0,0.0,0.0,0.0,0.0,0.0,SG,
2,SZ000001,平安银行,2025-01-02,SELL,11.73,11.73,NaN,1000.0,0.0,299800.0,0.0,0.0,0.0,0.0,0.0,SG,
3,SZ000001,平安银行,2025-02-11,BUY,11.40,11.40,NaN,1000.0,0.0,288400.0,0.0,0.0,0.0,0.0,0.0,SG,
4,SZ000001,平安银行,2025-02-27,SELL,11.53,11.53,NaN,1000.0,0.0,299930.0,0.0,0.0,0.0,0.0,0.0,SG,
5,SZ000001,平安银行,2025-02-28,BUY,11.58,11.58,NaN,1000.0,0.0,288350.0,0.0,0.0,0.0,0.0,0.0,SG,
6,SZ000001,平安银行,2025-03-03,SELL,11.52,11.52,NaN,1000.0,0.0,299870.0,0.0,0.0,0.0,0.0,0.0,SG,
7,SZ000001,平安银行,2025-03-06,BUY,11.69,11.69,NaN,1000.0,0.0,288180.0,0.0,0.0,0.0,0.0,0.0,SG,
8,SZ000001,平安银行,2025-03-19,SELL,11.48,11.48,NaN,1000.0,0.0,299660.0,0.0,0.0,0.0,0.0,0.0,SG,
9,SZ000001,平安银行,2025-04-21,BUY,11.00,11.00,NaN,1000.0,0.0,288660.0,0.0,0.0,0.0,0.0,0.0,SG,


In [4]:
#2017年2月21日以9.60的价格卖出100股
td = my_tm.sell(Datetime(201702210000), sm['sz000001'], 9.60)

my_tm

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-02-21 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current total funds: 100049.00,
  current cash: 100049.00,
  current market_value: 0.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
  Short Position: 
  Borrow Stock: 
}

# 2 利用 Excel 查看交易详情

使用 tocsv 方法将 TM 的交易记录、当前持仓及已平仓详细情况分别保存为 csv 文件，以便用 Excel 查看详情。

tocsv方法参数为一个指定的目录，目录必须以存在。其输出会在指定目录中，生成三个文件，“TM名称_交易记录.csv”、“TM名称_未平仓记录.csv”、“TM名称_已平仓记录.csv”。TM名称可在crtTM创建TM对象时指定，默认为“SYS”，如下图所示。

<img src="images/008_01_tocsv.png" align='left'>

In [5]:
#在 hikyuu_XXX.ini 文件中配置的临时路径中输出
my_tm.tocsv(sm.tmpdir())

使用 Excel 查看 csv，如：

<img src="images/008_02_tocsv_look.png" align="left">

# 3 使用序列化保存或重新载入已有TM对象

In [6]:
#保存至指定文件
from datetime import date
filename = "{}/my_trade/my_trade_record_{}.xml".format(sm.tmpdir(), date.today());
hku_save(my_tm, filename)

In [7]:
#载入已保存的TM对象
#filename = "{}/my_trade/my_trade_record_{}.xml".format(sm.tmpdir(), date.today())
new_my_tm = hku_load(filename)

使用 hku_save 保存的对象，其格式为XML文件，可直接使用 XML 工具或浏览器查看：

<img src="images/008_03_pickle.png" align="left">

# 4 使用订单代理

In [8]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(init_cash=300000, date=Datetime(201701010000))

#注册实盘交易订单代理
ob = crtOB(TestOrderBroker())
my_tm.reg_broker(ob) #TestOerderBroker是测试用订单代理对象，只打印
# 注意：pybind 不支持下面这种方式调用，必须先生成实例再传入！！！
# my_tm.reg_broker(crtOB(TestOrderBroker(), False))

#根据需要修改订单代理最后的时间戳，后续只有大于该时间戳时，订单代理才会实际发出订单指令
my_tm.broker_last_datetime=Datetime(201701010000)

#创建信号指示器（以5日EMA为快线，5日EMA自身的10日EMA作为慢线，快线向上穿越慢线时买入，反之卖出）
my_sg = SG_Flex(EMA(C, n=5), slow_n=10)

#固定每次买入1000股
my_mm = MM_FixedCount(1000)

#创建交易系统并运行
sys = SYS_Simple(tm = my_tm, sg = my_sg, mm = my_mm)
sys.run(sm['sz000001'], Query(-150))

买入：SZ000001, 价格: 11.93, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.73, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.4, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.53, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.58, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.52, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.69, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.48, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.0, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.0, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.07, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 12.46, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 12.48, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPa